# BN16 Hyperparameter Optimization

Playground for hyperparameter optimization for the 64-128-full-size CNN.

## Imports

In [1]:
#!TMPDIR=../../tmp pip install torch==1.7.* torch-summary numpy pandas matplotlib sparse ipywidgets
#!TMPDIR=../../tmp pip install 'ray[tune]'
!pip uninstall -y dataclasses

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary
import sparse
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import autocast

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from timeit import default_timer as timer
import datetime
import glob

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import gc

In [3]:
ray.init(num_gpus=2)
ray.get_gpu_ids()

2020-12-06 18:28:32,853	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


[]

## Config

In [4]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

print(DEVICE)
RND_SEED = 1
BATCH_SIZE = 8
N_STEPS = 12
N_INDIVIDUALS = 2443

torch.random.manual_seed(RND_SEED)
torch.cuda.manual_seed(RND_SEED)

pd.options.display.max_rows = 20
pd.options.display.min_rows = None
pd.options.display.width = 800
np.set_printoptions(edgeitems=5)
np.core.arrayprint._line_width = 300

cuda


## Additional Data

### Aliveness

In [5]:
observation_start_date = datetime.datetime(2016, 7, 23, 0, 0, 0, 0)
hatching_start_date = datetime.datetime(2016, 7, 19, 0, 0, 0, 0)

In [6]:
indices_data = pd.read_csv('indices_bn16.csv').sort_values(by='bee_id')
alive_data = pd.read_csv('alive_bn16.csv').sort_values(by='bee_id').reset_index(drop=True) # required or the data will be messed up

In [7]:
# combine both
combined_alive_data = alive_data
combined_alive_data['interaction_matrix_idx'] = indices_data['interaction_matrix_idx']
# now sort by matrix id for easy vector generation
combined_alive_data = combined_alive_data.sort_values(by='interaction_matrix_idx')
# date column type fix
combined_alive_data['date_emerged'] = pd.to_datetime(combined_alive_data['date_emerged'])
# let's translate the hatch day into a step where observation_start_date is step 0
combined_alive_data['born_in_step'] = combined_alive_data['date_emerged'].map(lambda x: (x - observation_start_date).days * 48)
combined_alive_data['dead_by_step'] = combined_alive_data.apply(lambda row: 48 * row['days_alive'] + row['born_in_step'], axis=1)
# combined_alive_data = get_combined_alive_data(indices_data, alive_data, observation_start_date)

For data insights like age distribution take a look at the `insights` notebook.

#### Util

In [8]:
def bee_id_to_matrix_id(bee_id, index_data):
    '''Translates the bee ID into the matrix ID.'''
    return indices_data[indices_data['bee_id'] == bee_id]['interaction_matrix_idx'].item()

def matrix_id_to_bee_id(matrix_id):
    '''Translates the matrix ID into the bee ID.'''
    return indices_data[indices_data['interaction_matrix_idx'] == matrix_id]['bee_id'].item()

def get_aliveness_vector(step):
    '''Returns a #individuals-long boolean vector of matrix-ID-indexed bee aliveness.'''
    return (combined_alive_data['born_in_step'] <= step) & (step < combined_alive_data['dead_by_step'])

def get_lifespan_for_row(row, step):
    if ((step < row['born_in_step']) | (row['dead_by_step'] < step)): #iloc[0] if you pass in a df
        return 0
    return row['dead_by_step'] - step  #iloc[0] if you pass in a df    

def get_lifespan_vector(step):
    '''Returns a #individuals-long vector of matrix-ID-indexed remaining bee lifespans in steps (half an hour).'''
    return combined_alive_data.apply(get_lifespan_for_row, axis=1, step=step)

def get_lifespan_for_individual(midx):
    '''Get the remaining steps to live for given individual matrix index.'''
    # start at zero, not before
    bis = max(combined_alive_data[combined_alive_data['interaction_matrix_idx'] == midx]['born_in_step'].item(), 0)
    dis = combined_alive_data.iloc[midx]['dead_by_step']
    # full range, 56 days à 48 steps
    r = np.zeros(56 * 48) 
    # dead in 4 steps => 4,3,2,1 - clip to length
    ins = np.arange(dis - bis)[::-1][:len(r) - bis] 
    r[bis:bis + len(ins)] = ins
    return r

### Circadian Rhythmicity

In [9]:
circadian_data = pd.read_csv(
    'rhythmicity_bn16.csv',
    dtype={'bee_id': np.int16, 'age': np.int8, 'date': str, 'circadian_rhythmicity': np.float64},
    parse_dates=['date'],
    date_parser = pd.to_datetime
)

In [10]:
# we need to expand the days into 48 copies, also, we don't know if there are gaps in the date so we should not rely on the date to be continous
def circadian_vector_for_individual(bee_id, circadian, observation_start_date):
    '''Get the vector of circadian rhythmicity per step for a bee.
    
    Parameters
    ----------
    bee_id : int
        The ID of the bee.
    circadian : pd.DataFrame
        The data frame with the circadian rhythmicities.
    observation_start_date: datetime
        The datetime of observation start.
    '''
    v = np.zeros(56 * 48)
    for row in circadian[circadian['bee_id'] == bee_id].itertuples():
        ins_start = (row.date - observation_start_date).days * 48
        v[ins_start:ins_start + 48] = np.repeat(row.circadian_rhythmicity, 48)
    return v

## Data Set

In [11]:
class IndividualDataSet(Dataset):
    '''Holds the data for one individual.'''
    def __init__(self, day, transform=None):
        self.data = sparse.load_npz('/home/mi/rbergmann/ma/bn16/individuals/{}.npz'.format(day))
        self.transform = transform
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        d = torch.from_numpy(self.data[idx].todense())
        if self.transform:
            d = self.transform(d)
        return d
    
class Bn16DataSet(Dataset):
    def __init__(self, days=(0, 56), transform=None):
        self.days = days
        self.transform = transform
        
    def __len__(self):
        return 2443
    
    def __getitem__(self, idx):        
        d = sparse.load_npz('/home/mi/rbergmann/ma/bn16/individuals/{}.npz'.format(idx))
        d = d[self.days[0] * 48:self.days[1] * 48]
        d = torch.from_numpy(d.todense().astype(np.float32))
        if self.transform:
            d = self.transform(d)
        return (
            d,
            {
                'lifespan': get_lifespan_for_individual(idx).astype('float'),
                'circadian': circadian_vector_for_individual(
                    matrix_id_to_bee_id(idx),
                    circadian_data,
                    observation_start_date
                ).astype('float')
            }
        )

### Split & Loaders

In [12]:
# first 36 days
bn16_data = Bn16DataSet(days=(0, 35))

In [13]:
# 80/20 split
train_data, test_data = random_split(bn16_data, [1955, 488])

full_loader = DataLoader(
    bn16_data
    ,batch_size=BATCH_SIZE
    ,shuffle=True
    ,num_workers=4
    ,drop_last=True
)
train_loader = DataLoader(
    train_data
    ,batch_size=BATCH_SIZE
    ,shuffle=True
    ,num_workers=4
    ,drop_last=True
)
test_loader = DataLoader(
    test_data
    ,batch_size=BATCH_SIZE
    ,shuffle=True
    ,num_workers=2
    ,drop_last=True
)

# Model

In [14]:
class CPC(nn.Module):
    def __init__(self, n_steps, batch_size, hidden_size=256, context_size=256):
        super(CPC, self).__init__()
        self.n_steps = n_steps
        self.hidden_size = hidden_size
        self.context_size = context_size
        self.g_enc = self.encoder()
        self.g_ar = self.autoregressive()
        self.step_W = nn.ModuleList([nn.Linear(self.context_size, self.hidden_size) for i in range(n_steps)])
        self.loss_criterion = nn.CrossEntropyLoss()
        self.batch_size = batch_size
        self.cpc_target = torch.arange(0, batch_size).long().to(DEVICE)

    def encoder(self):            
        # conv2d signature: in, out, kernel, stride, padding
        # input is: batch, channel, time, interactions
        return nn.Sequential(            
            # positional only : ~ 25% CPC
            nn.Conv2d(1, self.hidden_size, (1, 2443), 1, 0), # positional filter
            nn.ReLU(),
        )
        
    def autoregressive(self):
        return nn.GRU(self.hidden_size, self.context_size, batch_first=True)

    def predict(self, x, hidden=None):
        '''Return c_t for the whole given x, all its steps and the hidden state.'''
        with torch.no_grad():
            if hidden == None:
                hidden = self.empty_hidden()
            z = self.g_enc(x)
            z = z.squeeze().permute(0, 2, 1)
            pred, hidden = self.g_ar(z, hidden)
            return pred[:,-1,:], pred, hidden

#     @autocast()
    def forward(self, x, gar_hidden):
        # put through g_enc
        z = self.g_enc(x) # for convs
        # sample a subsequence of n steps
        # but we need at least one bit to feed into the GRU, so we pick from 1 to seqlen - n_steps
        subsample_idx = torch.randint(1, z.shape[1] - self.n_steps, size=(1,)) # 1-1680-12 = 1667
        
        # put everything before our pick through g_ar, collect c_t        
        # g_ar expects shape(batch, seq_len, input_size), h_0 of shape (num_layers * num_directions, batch, hidden_size)
        # https://pytorch.org/docs/master/generated/torch.nn.GRU.html#torch.nn.GRU
        # https://jdhao.github.io/2019/07/10/pytorch_view_reshape_transpose_permute/
        z = z.squeeze().permute(0, 2, 1)
        c_tx, gar_hidden = self.g_ar(z[:,:subsample_idx,:], gar_hidden)
        c_t = c_tx[:,-1,:].squeeze() # GRU outputs a c for every step - we want the last

        # for n steps, put through W_k for corresponding step to get predictions
        # W_k * c_t
        pred = [self.step_W[step](c_t) for step in range(self.n_steps)]

        # info nce loss
        acc = []
        info_nce_loss = []
        for time_step in range(self.n_steps):
            # f_k = exp(z_{t+k} * W_k * c_t)
            f_k = torch.mm(z[:,subsample_idx + time_step,:].squeeze(), torch.transpose(pred[time_step], 0, 1))
            info_nce_loss.append(self.loss_criterion(f_k, self.cpc_target))
            accuracy = torch.mean((torch.argmax(f_k, dim=1) == self.cpc_target).float())
            acc.append(accuracy)
        info_nce_loss = torch.stack(info_nce_loss).mean()
        
        return torch.mean(info_nce_loss), torch.mean(torch.tensor(acc))
    
    def empty_hidden(self):
        return torch.zeros(1, self.batch_size, self.context_size).to(DEVICE)

## Training Code

In [15]:
def train(config, checkpoint_dir='/home/mi/rbergmann/ma/bn16/'):
    data_loader = full_loader
    model = CPC(config['n_steps'], BATCH_SIZE, hidden_size=config['hidden_size'], context_size=config['context_size']).to(DEVICE)
    model.to(DEVICE)
    model.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=(0.9, 0.999), eps=config['eps'], weight_decay=0)
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    for epoch in range(50):
        loss = 0
        acc = 0
        epoch_loss = []
        epoch_acc = []
        
        for batch_idx, (x, y) in enumerate(data_loader):
            time_batch_start = timer()
            optimizer.zero_grad()

            # add channel dimension - for convolutions
            if len(x.shape) < 4:
                x = x.unsqueeze(1).to(DEVICE)

            # fwd - info_nce loss inside forward
            loss, accuracy = model(x, model.empty_hidden())

            # back
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())
            epoch_acc.append(accuracy)

        epoch_loss_mean = torch.mean(torch.tensor(epoch_loss, dtype=torch.float64)).item()
        epoch_acc_mean = torch.mean(torch.tensor(epoch_acc, dtype=torch.float64)).item()
        # epoch report
        print('Epoch: {} [{}/{} (100%)]\tEpoch Loss: {:.8f}\tEpoch Acc: {:.2f}%'.format(
            epoch,
            len(data_loader.dataset),
            len(data_loader.dataset),
            epoch_loss_mean,
            epoch_acc_mean * 100
        ))
        
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=epoch_loss[-1], accuracy=epoch_acc[-1])

# Training

## Training - CPC

Actual training of the CPC model.

In [16]:
config = {
    "hidden_size": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "context_size": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "n_steps": tune.choice(list(range(1, 17))),
    "lr": tune.loguniform(1e-4, 1e-1),
    #"betas": tune.loguniform(0.9, 0.9999),
    "eps": tune.loguniform(1e-09, 1e-06),
}
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=50,
    grace_period=1,
    reduction_factor=2
)
reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"]
)

In [ ]:
result = tune.run(
    train,
    resources_per_trial={"gpu": 2},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter,
    checkpoint_at_end=False, # happens within train
)

In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final loss: {}".format(best_trial.last_result["loss"]))
print("Best trial final accuracy: {}".format(best_trial.last_result["accuracy"]))

In [ ]:
best_trial.checkpoint.value

In [ ]:
ray.shutdown()
gc.collect()